### Problemes a resoudre dans ce DATASET

    1.Valeurs manquantes (NaN)
    2.  Doublons (clients en double)
    3.  Formats incohérents (dates, téléphones)
    4.  Casse incohérente (paris, PARIS, Paris)
    5.  Espaces en trop
    6.  Valeurs aberrantes (âge -5, âge 156)
    7.  Emails invalides
    8.  Catégories mal orthographiées
    9.  Booléens incohérents (oui/OUI/1/Oui)  
    10.  Montants négatifs ou irréalistes

In [5]:
# Chargement des données et exploration
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/clients_ecommerce_raw.csv')

print(df.head()) 
print(f"lignes et colonnes : {df.shape}")


   client_id             nom                      email       telephone  \
0          1   Martin Dupont    martin.dupont@gmail.com      0612345678   
1          2    marie claire      marie.claire@yahoo.fr  06 23 45 67 89   
2          3      JEAN PETIT       JEAN.PETIT@GMAIL.COM    +33634567890   
3          4   Sophie Martin  sophie.martin@hotmail.com      0645678901   
4          5  Pierre  Durand    pierre.durand@gmail.com  06.56.78.90.12   

     age      ville date_inscription  montant_achats  nb_commandes  \
0   25.0      Paris       2023-01-15          150.50           3.0   
1   34.0      paris       15/02/2023          230.00           5.0   
2   -5.0       LYON       2023-03-20          -50.00           0.0   
3   45.0  Marseille       20-04-2023           89.99           2.0   
4  156.0       Lyon       2023/05/10         1500.00          25.0   

  categorie_preferee newsletter  
0       Électronique        oui  
1               Mode        OUI  
2       electronique      

In [38]:
# Audit
# creation d'un rapport :

def audit(df):
    "Rapport du dataset: "
    rapport = pd.DataFrame({
        'Valeur Manquante' : df.isnull().sum(),
        'Doublons' : df.duplicated().sum(),
        'Types correct': df.dtypes,
    })
    return rapport


print(f"check les valeurs : \n {df.describe()}\n")
print(f"check les format incohérent : \n {df['ville'].unique()}\n")

audit(df)

check les valeurs : 
        client_id         age  montant_achats  nb_commandes
count  30.000000   26.000000       26.000000     26.000000
mean   15.033333   40.192308    38765.252308     11.846154
std     9.140600   26.561655   196054.458596     28.706365
min     1.000000   -5.000000      -50.000000      0.000000
25%     7.250000   29.250000      126.212500      3.000000
50%    14.500000   35.500000      232.250000      5.000000
75%    22.750000   44.750000      420.150000      8.750000
max    30.000000  156.000000   999999.990000    150.000000

check les format incohérent : 
 ['Paris' 'paris' 'LYON' 'Marseille' 'Lyon' 'Bordeaux' 'Toulouse' 'paris '
 ' Nantes' 'Strasbourg' 'Montpellier' 'bordeaux' 'Lille' 'Rennes' 'NICE'
 'Toulon' 'Grenoble' 'Dijon' 'Angers' 'Le Mans' nan 'Clermont' 'Brest'
 'Tours' 'Limoges' 'Amiens' 'Perpignan' 'Orléans']



,Valeur Manquante,Doublons,Types correct
client_id,0,1,int64
nom,2,1,object
email,2,1,object
telephone,3,1,object
age,4,1,float64
ville,2,1,object
date_inscription,1,1,object
montant_achats,4,1,float64
nb_commandes,4,1,float64
categorie_preferee,3,1,object
